**This notebook has been written to provide an overview of the kapow module and compare its usage to the standard openmm python library.**
We start by importing the required modules.

In [1]:
import kapow
from simtk import openmm

The openmm module is the standard python library shipped with the OpenMM simulation suite. The kapow module is provided by this package. Superficially the two should look more or less the same. For every class in openmm there is an equivalent class in kapow. **The kapow module aims to be an alternative interface to all of OpenMM's functionality.**  Lets take a closer look at how the modules interact with one another by creating a class instance. In openmm:

In [2]:
onbf = openmm.NonbondedForce()

we can then wrap this object with a kapow object:

In [3]:
knbf = kapow.NonbondedForce.Wrap(onbf)

The call to kapow.NonbondedForce.Wrap creates an instance that provides a different interface to the openmm instance. Any changes made to knbf will be reflect in knbf and vice versa. From the kapow instance you can access the original openmm instance as the wrapped_object attribute:

In [4]:
assert knbf.wrapped_object is onbf

Instead of doing this in two separate steps you can also call kapow.NonbondedForce() which will create a new openmm instance to wrap automatically. I've prefixed each of the instances with k and o for kapow and openmm respectively to distinguish between the two. To expose the differences lets look at how the attributes of these two differ. Ignoring magic methods:

In [5]:
print('Module - OpenMM')
for attr in dir(onbf):
    if not attr.startswith('_'):
        print(attr)

print('\nModule - Kapow')
for attr in dir(knbf):
    if not attr.startswith('_'):
        print(attr)

Module - OpenMM
CutoffNonPeriodic
CutoffPeriodic
Ewald
LJPME
NoCutoff
PME
addException
addException_usingRMin
addParticle
addParticle_usingRVdw
createExceptionsFromBonds
getCutoffDistance
getEwaldErrorTolerance
getExceptionParameters
getForceGroup
getLJPMEParameters
getLJPMEParametersInContext
getNonbondedMethod
getNumExceptions
getNumParticles
getPMEParameters
getPMEParametersInContext
getParticleParameters
getReactionFieldDielectric
getReciprocalSpaceForceGroup
getSwitchingDistance
getUseDispersionCorrection
getUseSwitchingFunction
setCutoffDistance
setEwaldErrorTolerance
setExceptionParameters
setForceGroup
setLJPMEParameters
setNonbondedMethod
setPMEParameters
setParticleParameters
setReactionFieldDielectric
setReciprocalSpaceForceGroup
setSwitchingDistance
setUseDispersionCorrection
setUseSwitchingFunction
this
updateParametersInContext
usesPeriodicBoundaryConditions

Module - Kapow
CutoffNonPeriodic
CutoffPeriodic
Ewald
LJPME
LJPMEParameters
NoCutoff
PME
PMEParameters
Wrap
addExc

Notice the pattern? The kapow class has far fewer attributes, it also doesn't have any attributes that begin with get or set. Instead for every quantity controlled by a get and set attribute in openmm there is a single attribute in kapow. As a simple example, for the potential cutoff distance the openmm instance uses:

In [6]:
print('cutoff', onbf.getCutoffDistance())
onbf.setCutoffDistance(0.8)
print('cutoff', onbf.getCutoffDistance())

cutoff 1.0 nm
cutoff 0.8 nm


Whilst the kapow instance uses:

In [7]:
print('cutoff', knbf.cutoffDistance)
knbf.cutoffDistance = 1.0
print('cutoff', knbf.cutoffDistance)

cutoff 0.8 nm
cutoff 1.0 nm


The difference in the code here again is fairly small, at least for this simple example. Usage of the kapow interface is slightly more clean and compact, but more important is the principle that using get and set methods is considered poor practice in Python (see [here](https://eli.thegreenplace.net/2009/02/06/getters-and-setters-in-python) for a discussion on why). Kapow has taken the methods 'getCutoffDistance' and 'setCutoffDistance' and replaced them with a single property. This is a straightforward case where the get and set methods deal with a single value. Commonly however get and set methods are used to manage access to array elements requiring more sophisticated treatment.

In addition to get and set methods, to manage arrays we also need to be able to determine the number of elements of the array and, optionally, have routines to add and remove elements. As an example lets look at particles for the NonbondedForce. In the openmm NonbondedForce we have the below methods:

In [8]:
onbf.getNumParticles
onbf.addParticle
onbf.getParticleParameters
onbf.setParticleParameters

<bound method NonbondedForce.setParticleParameters of <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f27fbd74a50> >>

This is replaced by a single attribute in the kapow class:

In [9]:
print(knbf.particles)
type(knbf.particles)

<Wrapped C-array containing 0 objects>


kapow.wrappers.ArrayWrapper

knbf.particles is a custom object provided by Kapow that can be thought of as an extension to the Python property object. In short, it provides an interface which looks and behaves a lot like a Python list but which under the hood is calling the above methods of onbf. A practical example is probably best. Before doing anything interesting we need to add some particles to the NonbondedForce. With the openmm interface:

In [10]:
onbf.addParticle(1., 1., 1.)

0

and the equivalent call for kapow:

In [11]:
knbf.particles.append((1., 1., 1.))

1

As onbf and knbf are different interfaces for the same object, we should now have two particles. We can check this with:

In [12]:
print(onbf.getNumParticles())
# vs
print(len(knbf.particles))


2
2


Because knbf.particles attempts to behave like a list we can ask for its length. In this case, the \_\_len\_\_ method of knbf.particles is simply wrapping a call to onbf.getNumparticles(). Intuitively, we can also index knbf.particles:

In [13]:
p = knbf.particles[0]
print(p)
print(type(p))

Particle(charge=Quantity(value=1.0, unit=elementary charge), sigma=Quantity(value=1.0, unit=nanometer), epsilon=Quantity(value=1.0, unit=kilojoule/mole))
<class 'kapow.wrappers.Particle'>


Now we see something interesting! Compare this with the openmm object.

In [14]:
print(onbf.getParticleParameters(0))

[Quantity(value=1.0, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=1.0, unit=kilojoule/mole)]


Rather than returning a simple list, indexing knbf.particles returns a custom Particle object. The Particle object was constructed automatically based on the parameters of the addParticle method. This helps us by providing a much more informative return value and some other handy features as we'll see shortly. The Particle object is derived from a [namedtuple](https://docs.python.org/2/library/collections.html#collections.namedtuple) so you can always use a tuple (or any iterable) in its place meaning this approach also maintains a good level of flexibility.

Now my favorite feature. You can iterate a list, so you can iterate knbf.particles:

In [15]:
for p in knbf.particles:
    print(p)

Particle(charge=Quantity(value=1.0, unit=elementary charge), sigma=Quantity(value=1.0, unit=nanometer), epsilon=Quantity(value=1.0, unit=kilojoule/mole))
Particle(charge=Quantity(value=1.0, unit=elementary charge), sigma=Quantity(value=1.0, unit=nanometer), epsilon=Quantity(value=1.0, unit=kilojoule/mole))


The equivalent code for this with the original interface is quite hideous from a pythonic point of view:

In [16]:
for i in range(onbf.getNumParticles()):
    print(onbf.getParticleParameters(i))

[Quantity(value=1.0, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=1.0, unit=kilojoule/mole)]
[Quantity(value=1.0, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=1.0, unit=kilojoule/mole)]


Finally, an example combining iteration with the ability to set elements:

In [17]:
for i, p in enumerate(knbf.particles):
    knbf.particles[i] = p._replace(sigma=2.)

Here, \_replace is a handy method of namedtuples. The equivalent openmm code is much less compact and readable:

In [18]:
for i in range(onbf.getNumParticles()):
    charge, sigma, epsilon = onbf.getParticleParameters(i)
    onbf.setParticleParameters(i, charge, 2., epsilon)

ArrayWrappers such as knbf.particles also have detailed help strings that describe the methods that they wrap

In [19]:
help(knbf.particles)

Help on ArrayWrapper in module kapow.wrappers object:

class ArrayWrapper(ArrayWrapper)
 |  This object provides a pythonic interface to an underlying C++ array,
 |  mimicing the semantics of a list. This object has methods and attributes that
 |  wrap calls to the openmm base class, <class 'simtk.openmm.openmm.NonbondedForce'>.
 |  This object can be indexed to call the get and set routines detailed below.
 |  Where possible, accessed array elements will be provided as a namedtuple
 |  to give additional information about the underlying data.
 |  This function may also have append and pop attributes to add or remove
 |  elements of the array.
 |  
 |  This object provides a list-like interface that wraps the below
 |  function calls from the openmm base class:
 |  Wrapped length function - getNumParticles
 |  Wrapped get functions - ['getParticleParameters']
 |  Wrapped set functions - ['setParticleParameters']
 |  Wrapped add function  - addParticle
 |  Wrapped remover function - Non

In addition to ArrayWrappers, another core piece of functionality from the kapow module is the DictWrapper. ArrayWrappers allow a neat way to simpify underlying C++ structures that support indexing. DictWrappers expand this to include key-value pair data structures. This is particularly useful for classes using names for attributes. Consider the below code to examine the default global variables for an AMDIntegrator with openmm:

In [20]:
amd = openmm.AMDIntegrator(0., 0, 0)
for i in range(amd.getNumGlobalVariables()):
    print(amd.getGlobalVariableName(i), amd.getGlobalVariable(i))

alpha 0.0
E 0.0


Each global variable has a name but getting hold of that name and the value that goes along with it is rather laborious. In the kapow module these are instead treated as key-value pairs in a dictionary-like interface:

In [21]:
amd = kapow.AMDIntegrator(0., 0., 0)
print(amd.globalVariables.items())

(('alpha', 0.0), ('E', 0.0))
